In [1]:
# # 선호장르와 일반장르를 구분하여 MMR 검색을 수행하는 함수
# def genre_weighted_mmr_search(db, query, preferred_genre, k=2):
#     # MMR 검색 설정
#     retriever = db.as_retriever(
#         search_type="mmr",
#         search_kwargs={
#             "k": k,  # 최종 반환할 문서 수
#             "fetch_k": 10,  # 초기 검색할 문서 수
#             "lambda_mult": 0.6  # 다양성 vs 관련성 가중치
#         }
#     )
    
#     # 장르별 필터 설정
#     preferred_filter = {"metadata": {"genre": preferred_genre}}
#     general_filter = {"metadata": {"genre": {"$ne": preferred_genre}}}
    
#     # 선호장르(60%)와 일반장르(40%) 문서 수 계산
#     preferred_count = int(k * 0.6)
#     general_count = k - preferred_count
    
#     # 선호장르 문서 검색
#     preferred_docs = retriever.get_relevant_documents(
#         query,
#         filter=preferred_filter,
#         k=preferred_count
#     )
    
#     # 일반장르 문서 검색
#     general_docs = retriever.get_relevant_documents(
#         query,
#         filter=general_filter,
#         k=general_count
#     )
    
#     # 결과 합치기
#     combined_docs = preferred_docs + general_docs
    
#     return combined_docs

# # 사용 예시
# preferred_genre = "SF"  # 선호하는 장르
# results = genre_weighted_mmr_search(db, "임베딩(Embedding)은 무엇인가요?", preferred_genre)

# # 결과 출력
# for doc in results:
#     print(doc.page_content)
#     print("=========================================================")

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

In [41]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader

####################
####### RAG 챗봇 구축
###################

# 1. LLM 모델 불러오기
llm = ChatOpenAI(model="gpt-4o-mini")  # GPT-4o-mini 모델을 사용하여 LLM을 초기화합니다.

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")  # 텍스트 임베딩 모델을 설정합니다.

# 저장된 데이터 로드
vectorstore = FAISS.load_local(
    folder_path="faiss_db",  # FAISS 데이터베이스가 저장된 폴더 경로
    index_name="index",  # 사용할 인덱스 이름
    embeddings=embeddings,  # 임베딩 모델
    allow_dangerous_deserialization=True,  # 위험한 역직렬화를 허용합니다.
)

db = vectorstore  # 데이터베이스 객체를 설정합니다.

def genre_weighted_mmr_search(db, query, preferred_genre, k=20):
    # MMR 검색을 위한 리트리버 설정
    retriever = db.as_retriever(
        search_type="mmr",  # MMR 검색 방식
        search_kwargs={
            "k": k,  # 검색할 문서 수
            "fetch_k": 10,  # 추가로 가져올 문서 수
            "lambda_mult": 0.1  # MMR의 람다 값
        }
    )
    
    # 선호장르(60%)와 일반장르(40%) 문서 수 계산
    preferred_count = int(k * 0.6)  # 선호 장르 문서 수
    general_count = k - preferred_count  # 일반 장르 문서 수
    
    # 선호 장르 필터 설정
    preferred_filter = {
        "genre": {"$eq": preferred_genre}  # 선호 장르에 대한 필터
    }
    
    # 선호 장르 문서 검색
    preferred_docs = retriever.get_relevant_documents(
        query,
        filter=preferred_filter,
        k=preferred_count  # 선호 장르 문서 수만큼 검색
    )
    
    # 일반 장르 문서 검색: 필터 없이 호출 후 직접 필터링
    all_docs = retriever.get_relevant_documents(
        query,
        k=general_count + 10  # 여유를 두고 추가 문서를 가져옵니다.
    )
    # 일반 장르 문서 필터링
    general_docs = [doc for doc in all_docs if doc.metadata.get("genre") != preferred_genre][:general_count]
    
    return preferred_docs + general_docs  # 선호 장르와 일반 장르 문서 결합하여 반환

# 7. 프롬프트 템플릿 구축하기 (물어온 데이터로)
prompt = ChatPromptTemplate.from_template("""
너는 {personality} 성격의 영화 평론가야. 그리고 너는 영화를 추천해주는 역할이야. \
답변을 끝낸 문장 끝에 '이 정도면 적절하게 추천해드린 것 같거덩요'를 붙여야 해. \
서술어로는 '~거덩요'로 완성해서 대답해줘. \
나의 말에 대답하도록 해. \
오로지 아래의 context만을 기반으로 질문에 대답하세요:
{context}
질문:
{question} """)

personality = "굉장히 나이스하고 친절한"  # 영화 평론가의 성격 설정
question = "Deadpool & Wolverine 영화 정보좀 알려줘"  # 질문 설정

# 8. 1~7의 요소들을 chain으로 조합하여 RAG 구축 완료
def format_docs(docs):
    # 필요에 따라 docs의 데이터 구조 검사 후 포맷팅
    return "\n\n".join(doc.page_content for doc in docs)  # 문서 내용을 포맷팅하여 반환

def get_retrieval_chain(query, personality_value):
    # 문서 검색 및 포맷팅
    retrieved_docs = genre_weighted_mmr_search(db, query, "action")  # 장르 문서 검색
    formatted_docs = format_docs(retrieved_docs)  # 검색된 문서 포맷팅

    # 체인 입력 설정
    chain_inputs = {
        "context": formatted_docs,  # 포맷팅된 문서
        "question": query,  # 질문
        "personality": personality_value  # 성격
    }
    print(formatted_docs)
    print("-" * 50)
    result = llm.invoke(prompt.format(**chain_inputs))  # LLM에 입력하여 결과 얻기
    return result.content  # 결과 반환

####################
####### 구축한 RAG 챗봇 실행
###################

response = get_retrieval_chain(question, personality)  # 챗봇 실행
print(response)  # 결과 출력

Marvel Studios Assembled: The Making of Deadpool & Wolverine Hugh Jackman, Ryan Reynolds, and director Shawn Levy sit down and spill the proverbial beans about how "Deadpool & Wolverine" was conceived, carried to term, and then birthed. Documentary making of, marvel cinematic universe (mcu)

가문의 영광 The film is a gangster comedy about a businessman who becomes involved with the gangster underworld through the daughter of a crime boss. Comedy, Action, Crime gangster, marriage, family

The Wolverine Wolverine faces his ultimate nemesis - and tests of his physical, emotional, and mortal limits - in a life-changing voyage to modern-day Japan. Action, Science Fiction, Adventure, Fantasy japan, samurai, world war i, superhero, mutant, based on comic, superhuman, duringcreditsstinger

Deadpool 2 Wisecracking mercenary Deadpool battles the evil and powerful Cable and other bad guys to save a boy's life. Action, Comedy, Adventure hero, superhero, mutant, mercenary, based on comic, sequel, afterc